<a href="https://colab.research.google.com/github/alfa7g7/Analisis-Cuantitativo-1er-semestre/blob/main/Examen_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FACULTAD DE INGENIERÍA, DISEÑO Y CIENCIAS APLICADAS MAESTRÍA EN CIENCIA DE DATOS TIC 60153 – Análisis Cuantitativo Examen 2 (50 %): Modelos de Regularización

### Ejercicio 1. 1)	Considere el conjunto de datos “Boston Housing Data” presentados en Harrison and Rubinfeld (1978). Defina como variable respuesta a la columna MEDV. Realice una partición 80-20, donde el primer 80 % de los datos son datos de entrenamiento y el restante 20 % son datos para prueba.


- Genere los modelos de regresión por regularización **Ridge, LASSO y Elastic-Net** para los datos de entrenamiento. Encuentre los valores óptimos de $α$ y $λ$ junto a su respectiva gráfica de evolución de los coeficientes de regresión. Compare los modelos en términos de la selección de variables, interprete los coeficientes y escriba la ecuación ajustada de regresión para cada caso. Finalmente, realice una predicción con las observaciones de prueba y determine cual de los tres modelos es el mejor en capacidad predictiva **(RMSE)**.
- Sea $Z ∈ \mathbb{R}^{n \times p}$ una muestra de datos multivariantes. La distancia de Mahalanobis (MD), presentado por Mahalanobis (1936) y conocida como la distancia estadística, se define de la siguiente forma:
$$MD(zi) = \sqrt{(zi − \hatµ)'\hat{∑}^{-1}(zi - \hat{µ})},\  para\ i=1, . . . , n.$$
Donde $xi ∈ \mathbb{R}^{p\times1}$ denota una observación (fila), pero transpuesta, de la muestra, $\hat{µ} ∈ \mathbb{R}^{p\times1}$ el vector de medias (vector con los promedios de cada columna o variable) y $\hat{Σ}^{−1} ∈ \mathbb{R}^{p\times p}$ la inversa de la matriz de covarianza de los datos. MD puede interpretarse como la distancia
que tiene cada individuo de la muestra al individuo ideal o equilibrio (representado por $\hat{µ}$), considerando la estructura de dependencia de las variables.

  Esta distancia tiene múltiples usos, como por ejemplo la detección de observaciones atípicas. Se dice que una observación $zi$ es atípica sí $MD(zi)> \chi^{2}_{(q,p)},$ donde q denota un percentil, usualmente $q ∈ [0.95, 0.99]$, de la distribución Chi-squared de parámetro $p$.
  
  De acuerdo a la información anterior: Detecte las observaciones atípicas multivariantes de la muestra, usando $q = 0.90, 0.95, 0.975, 0.99$ y en un gráfico muestre las distancias calculadas para cada individuo y los puntos de corte $(\chi^{2}_{(q,p)})$ ¿Qué puede concluir acerca de los individuos atípicos para cada punto de corte? ¿Quienes son? ¿El gráfico de distancias muestra alguna observación NO atípica que para usted si lo fue? ¿Qué puede concluir acerca del comportamiento de q en el valor del punto de corte? Justifique detalladamente.

- Realice dos modelos de regresión Robusto–Regularizado, combinando la técnica de detección de atípicos multivariantes más la regularización Elastic-Net. Encuentre los valores óptimos de $α^{*}$ y $λ^{*}$, interprete los coeficientes de regresión, escriba la respectiva ecuación de regresión resultante y valide los supuestos del modelo $(ε_i \ iid∼   N(0, σ^{2}))$. Finalmente, realice una predicción con las observaciones de prueba y determine el **RMSE** de estos dos modelos. Compare los resultados con los modelos estimados en el ítem 1.

  NOTA: La aplicación de la técnica de detección de atípicos debe realizarla de la siguiente manera:
  
  - Modelo 1: Estandarizar las variables y calcular MD usando el vector de medianas en vez del vector de medias y la matriz de correlación de Spearman en vez de la matriz de covarianzas usual. Luego, eliminar los atípicos y con la muestra limpia estimar el modelo regularizado.

  - Modelo 2: Estandarizar las variables y calcular MD usando el vector de medianas en vez del vector de medias y la matriz de correlación de Spearman en vez de la matriz de covarianzas usual. Luego, determinar como observaciones NO atípicas los primeros r n+p+1 l individuos con las distancias más pequeñas. Finalmente, con la muestra limpia estimar el modelo regularizado.
  
  Ejemplo: Si $n = 1000$ y $p = 100$ entonces r n+p+1 l = r 1000+100+1 l = r550.5l = 551. 2	2 Es decir, debe identificar como NO atípicos los 551 individuos de la muestra con las distancias de Mahalanobis más pequeñas.


#### Librerias a importar

In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle
from itertools import combinations

#### Consideraciones

The Boston house-price data of Harrison, D. and Rubinfeld, D.L. 'Hedonic
prices and the demand for clean air', J. Environ. Economics & Management,
vol.5, 81-102, 1978.   Used in Belsley, Kuh & Welsch, 'Regression diagnostics
...', Wiley, 1980.   N.B. Various transformations are used in the table on
pages 244-261 of the latter.

Variables in order:


CRIM     per capita crime rate by town


ZN       proportion of residential land zoned for lots over 25,000 sq.ft.


INDUS    proportion of non-retail business acres per town


CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)


NOX      nitric oxides concentration (parts per 10 million)


RM       average number of rooms per dwelling


AGE      proportion of owner-occupied units built prior to 1940


DIS      weighted distances to five Boston employment centres


RAD      index of accessibility to radial highways


TAX      full-value property-tax rate per $10,000


PTRATIO  pupil-teacher ratio by town


B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town


LSTAT    % lower status of the population


MEDV     Median value of owner-occupied homes in $1000's


In [85]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None,
  names=["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV"])

In [86]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012 entries, 0 to 1011
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     1012 non-null   float64
 1   ZN       1012 non-null   float64
 2   INDUS    1012 non-null   float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        0 non-null      float64
 12  LSTAT    0 non-null      float64
 13  MEDV     0 non-null      float64
dtypes: float64(14)
memory usage: 110.8 KB


In [87]:
#No al racismo jamás
raw_df = raw_df.drop(columns =["B"])

In [88]:
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

In [89]:
raw_df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
0,0.00632,18.00,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,NaN,NaN
1,396.90000,4.98,24.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.02731,0.00,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,NaN,NaN
3,396.90000,9.14,21.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.02729,0.00,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,396.90000,5.64,23.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008,0.10959,0.00,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,NaN,NaN
1009,393.45000,6.48,22.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010,0.04741,0.00,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,NaN,NaN


In [92]:
df = pd.DataFrame(data)

In [93]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,NaN,NaN,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,NaN,NaN,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,NaN,NaN,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,NaN,NaN,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,NaN,NaN,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,NaN,NaN,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,NaN,NaN,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,NaN,NaN,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,NaN,NaN,393.45,6.48
